In [1]:
import numpy as np
import pandas as pd
import datetime
import pickle #pickle序列化对象并保存到磁盘中，并在需要的时候读取出来
import os

In [2]:
############读取所有的文件#########################
def read_files():
    #用户表
    df_user = pd.read_csv("JData_User.csv")
    #商品表
    df_product = pd.read_csv("JData_Product.csv")
    #评论表
    df_comment = pd.read_csv("JData_Comment.csv")
    #行为表
    df_action_1 = pd.read_csv("JData_Action_201602.csv")
    df_action_2 = pd.read_csv("JData_Action_201603.csv")
    df_action_3 = pd.read_csv("JData_Action_201604.csv")
    df_action = pd.concat([df_action_1, df_action_2, df_action_3], axis=0)
        
    return df_user, df_product, df_comment, df_action

df_user, df_product, df_comment, df_action = read_files()

In [5]:
def convert_type(df_user, df_product, df_comment, df_action):
    print(df_user.dtypes)
    print(df_user.head())
    print("----------------")
    print(df_product.dtypes)
    print(df_product.head())
    print("----------------")
    print(df_comment.dtypes)
    print(df_comment.head())
    print("----------------")
    print(df_action.dtypes)
    print(df_action.head())
    print("----------------")
    
    print("processing df_user...")
    df_user.loc[:, ["age", "sex", "user_lv_cd"]] = df_user.loc[:, ["age", "sex", "user_lv_cd"]].fillna(-1)
    
    df_user.loc[:, "sex"] = df_user.loc[:, "sex"].apply(lambda x: -1 if int(x) == 2 else x)
    
    df_user.loc[:, "user_reg_tm"] = df_user.loc[:, "user_reg_tm"].apply(lambda x : pd.to_datetime(x))
    df_user.loc[:, ["user_id", "age", "sex", "user_lv_cd"]] \
        = df_user.loc[:, ["user_id", "age", "sex", "user_lv_cd"]].astype(np.int32)
    
    print("processing df_product...")
    df_product.loc[:, ["a1", "a2", "a3"]] = df_product.loc[:, ["a1", "a2", "a3"]].fillna(-1)
    
    df_product.loc[:, ["sku_id", "a1", "a2", "a3", "cate", "brand"]] \
        = df_product.loc[:, ["sku_id", "a1", "a2", "a3", "cate", "brand"]].astype(np.int32)
    
    print("processing df_comment...")
    df_comment.loc[:, ["comment_num", "has_bad_comment", "bad_comment_rate"]] \
        = df_comment.loc[:, ["comment_num", "has_bad_comment", "bad_comment_rate"]].fillna(0)
    
    df_comment.loc[:, "dt"] = df_comment.loc[:, "dt"].apply(lambda x : pd.to_datetime(x))
    df_comment.loc[:, ["sku_id", "comment_num", "has_bad_comment"]] \
        = df_comment.loc[:, ["sku_id", "comment_num", "has_bad_comment"]].astype(np.int32)
    df_comment.loc[:, "bad_comment_rate"] = df_comment.loc[:, "bad_comment_rate"].astype(np.float32)
    
    print("processing df_action...")
    df_action.loc[:, "time"] = df_action.loc[:, "time"] \
        .apply(lambda x : datetime.datetime.strptime(str(x), "%Y-%m-%d %H:%M:%S"))
    df_action.loc[:, ["user_id", "sku_id", "type", "cate", "brand"]] \
        = df_action.loc[:, ["user_id", "sku_id", "type", "cate", "brand"]].astype(np.int32)
    
    print("################")
    print(df_user.dtypes)
    print(df_user.head())
    print("----------------")
    print(df_product.dtypes)
    print(df_product.head())
    print("----------------")
    print(df_comment.dtypes)
    print(df_comment.head())
    print("----------------")
    print(df_action.dtypes)
    print(df_action.head())
    print("----------------")
    
    return df_user, df_product, df_comment, df_action
    
    
df_user, df_product, df_comment, df_action = convert_type(df_user, df_product, df_comment, df_action)

user_id                 int32
age                     int32
sex                     int32
user_lv_cd              int32
user_reg_tm    datetime64[ns]
dtype: object
   user_id  age  sex  user_lv_cd user_reg_tm
0   200001    6   -1           5  2016-01-26
1   200002   -1    0           1  2016-01-26
2   200003    4    1           4  2016-01-26
3   200004   -1   -1           1  2016-01-26
4   200005    2    0           4  2016-01-26
----------------
sku_id    int32
a1        int32
a2        int32
a3        int32
cate      int32
brand     int32
dtype: object
   sku_id  a1  a2  a3  cate  brand
0      10   3   1   1     8    489
1  100002   3   2   2     8    489
2  100003   1  -1  -1     8     30
3  100006   1   2   1     8    545
4   10001  -1   1   2     8    244
----------------
dt                  datetime64[ns]
sku_id                       int32
comment_num                  int32
has_bad_comment              int32
bad_comment_rate           float32
dtype: object
          dt  sku_id  c

In [39]:
def save_data(df_user, df_product, df_comment, df_action):
    if not os.path.exists("data.pkl"):
        with open("data.pkl", "wb") as f:
            pickle.dump((df_user, df_product, df_comment, df_action), f)
            
def load_data():
    with open("data.pkl", "rb") as f:
        df_user, df_product, df_comment, df_action = pickle.load(f)
    return df_user, df_product, df_comment, df_action

# save_data(df_user, df_product, df_comment, df_action)
df_user, df_product, df_comment, df_action = load_data()

In [40]:
def get_user_reg_before_416(df_user):
    return df_user.loc[df_user.loc[:, "user_reg_tm"] < pd.to_datetime("2016-04-16")]

df_user_reg_before_416 = get_user_reg_before_416(df_user)

In [41]:
def del_user_in_df_user(df_user, df_user_reg_before_416):
    df_user = df_user.loc[df_user.loc[:, "user_id"].isin(df_user_reg_before_416.loc[:, "user_id"])]
    return df_user

df_user = del_user_in_df_user(df_user, df_user_reg_before_416)

In [42]:
#获取商品评论的特征，以最近的日期为准
def get_df_last_comment(df_comment):
    df_comment = df_comment.loc[df_comment.loc[:, "dt"] < pd.to_datetime("2016-04-16")]
    df_time = df_comment.loc[:, ["sku_id", "dt"]]
    df_last_time = df_time.groupby("sku_id").max().reset_index()
    df_last_comment = pd.merge(df_comment, df_last_time, how="inner", on=["sku_id", "dt"])
    df_last_comment.pop("dt")
    return df_last_comment

def combine_df_product_comment(df_product, df_last_comment):
    df_sku = pd.merge(df_product, df_last_comment, how="left", on="sku_id").fillna(0)
    df_sku.loc[:, ["comment_num", "has_bad_comment"]] = df_sku.loc[:, ["comment_num", "has_bad_comment"]].astype(np.int32)

#     df_sku.loc[:, "comment_num"] = df_sku.loc[:, "comment_num"].apply(lambda x : int(x) if not np.isnan(x) else 0)
#     df_sku.loc[:, "has_bad_comment"] = df_sku.loc[:, "has_bad_comment"].apply(lambda x : int(x) if not np.isnan(x) else 0)
#     df_sku.loc[:, "bad_comment_rate"] = df_sku.loc[:, "bad_comment_rate"].apply(lambda x : 0.0 if np.isnan(x) else x)
    return df_sku

df_last_comment = get_df_last_comment(df_comment)
df_sku = combine_df_product_comment(df_product, df_last_comment)
print(df_sku.head())

   sku_id  a1  a2  a3  cate  brand  comment_num  has_bad_comment  \
0      10   3   1   1     8    489            0                0   
1  100002   3   2   2     8    489            0                0   
2  100003   1  -1  -1     8     30            0                0   
3  100006   1   2   1     8    545            0                0   
4   10001  -1   1   2     8    244            0                0   

   bad_comment_rate  
0               0.0  
1               0.0  
2               0.0  
3               0.0  
4               0.0  


In [43]:
#删除所有时间段都没有购买行为的用户
def get_df_buy_user(df_action):
    df_buy_action = df_action.loc[df_action.loc[:, "type"] == 4]
    df_buy_user = df_buy_action.loc[:, "user_id"].drop_duplicates()
    return df_buy_user

def del_no_buy_user(df_user, df_buy_user):
    print(len(df_user))
    print(len(df_buy_user))
#     df_user = pd.merge(df_user, df_buy_user, how="right", on="user_id")
    df_user = df_user.loc[df_user.loc[:, "user_id"].isin(df_buy_user)]
    print(len(df_user))
    return df_user
    
df_buy_user = get_df_buy_user(df_action)
df_user = del_no_buy_user(df_user, df_buy_user)
print(df_user.head())

105309
29485
29483
    user_id  age  sex  user_lv_cd user_reg_tm
0    200001    6   -1           5  2016-01-26
4    200005    2    0           4  2016-01-26
13   200014    3   -1           4  2013-04-10
14   200015    3    1           3  2016-01-26
16   200017    4   -1           4  2016-01-26


In [44]:
def del_user_sku_action_not_in_each_other(df_user, df_sku, df_action):
    df_user_intersection = pd.merge(df_user.loc[:, "user_id"], df_action.loc[:, "user_id"], how="inner", on="user_id")
    df_sku_intersection = pd.merge(df_sku.loc[:, "sku_id"], df_action.loc[:, "sku_id"], how="inner", on="sku_id")
    
    df_action = df_action.loc[(df_action.loc[:, "sku_id"].isin(df_sku_intersection.loc[:, "sku_id"])) \
                              & (df_action.loc[:, "user_id"].isin(df_user_intersection.loc[:, "user_id"]))]
    df_user = df_user.loc[df_user.loc[:, "user_id"].isin(df_action.loc[:, "user_id"])]
    df_sku = df_sku.loc[df_sku.loc[:, "sku_id"].isin(df_action.loc[:, "sku_id"])]
    return df_user, df_sku, df_action
df_user, df_sku, df_action = del_user_sku_action_not_in_each_other(df_user, df_sku, df_action)
print(df_user.head())
print(df_sku.head())
print(df_action.head())

    user_id  age  sex  user_lv_cd user_reg_tm
0    200001    6   -1           5  2016-01-26
4    200005    2    0           4  2016-01-26
13   200014    3   -1           4  2013-04-10
14   200015    3    1           3  2016-01-26
16   200017    4   -1           4  2016-01-26
    sku_id  a1  a2  a3  cate  brand  comment_num  has_bad_comment  \
16  100119   2   1   2     8    812            1                0   
27  100174   2   2   1     8    812            2                0   
40  100310   3   2  -1     8    214            0                0   
45  100344   3   1   1     8    214            0                0   
69  100462   3   1   2     8    214            3                0   

    bad_comment_rate  
16               0.0  
27               0.0  
40               0.0  
45               0.0  
69               0.0  
    user_id  sku_id                time  model_id  type  cate  brand
0    266079  138778 2016-01-31 23:59:02       NaN     1     8    403
1    266079  138778 2016-01-31 23

In [45]:
def del_useless_columns(df_user, df_sku, df_action):
    print(df_user.columns)
    print(df_sku.columns)
    print(df_action.columns)
    
    df_user.pop("user_reg_tm")
    df_action.pop("model_id")
    df_action.pop("cate")
    df_action.pop("brand")
    
    return df_user, df_sku, df_action
df_user, df_sku, df_action = del_useless_columns(df_user, df_sku, df_action)
print(df_user.head())
print(df_sku.head())
print(df_action.head())

Index(['user_id', 'age', 'sex', 'user_lv_cd', 'user_reg_tm'], dtype='object')
Index(['sku_id', 'a1', 'a2', 'a3', 'cate', 'brand', 'comment_num',
       'has_bad_comment', 'bad_comment_rate'],
      dtype='object')
Index(['user_id', 'sku_id', 'time', 'model_id', 'type', 'cate', 'brand'], dtype='object')
    user_id  age  sex  user_lv_cd
0    200001    6   -1           5
4    200005    2    0           4
13   200014    3   -1           4
14   200015    3    1           3
16   200017    4   -1           4
    sku_id  a1  a2  a3  cate  brand  comment_num  has_bad_comment  \
16  100119   2   1   2     8    812            1                0   
27  100174   2   2   1     8    812            2                0   
40  100310   3   2  -1     8    214            0                0   
45  100344   3   1   1     8    214            0                0   
69  100462   3   1   2     8    214            3                0   

    bad_comment_rate  
16               0.0  
27               0.0  
40      

In [48]:
print(len(df_user))
print(len(df_sku))
print(len(df_action))

29430
3403
6589539


In [54]:
def save_data(df_user, df_sku, df_action, filename):
    if not os.path.exists(filename):
        with open(filename, "wb") as f:
            pickle.dump((df_user, df_sku, df_action), f)
            
def load_data(filename):
    with open(filename, "rb") as f:
        df_user, df_sku, df_action = pickle.load(f)
    return df_user, df_sku, df_action

# save_data(df_user, df_sku, df_action, "clean_data_without_no_buy.pkl")
# df_user, df_sku, df_action = load_data()

In [53]:
!ls -a

.			 JData_User.csv		   df_dataset.pkl
..			 catboost_info		   jdata.7z
.ipynb_checkpoints	 clean_data.pkl		   model.cb
JData_Action_201602.csv  columns.txt		   terminal.ipynb
JData_Action_201603.csv  data.pkl		   test.ipynb
JData_Action_201604.csv  data_clean.ipynb	   京东购买预测数据集
JData_Comment.csv	 data_gen_feature.ipynb
JData_Product.csv	 data_visualization.ipynb


In [29]:
df_user.dtypes

user_id       int32
age           int32
sex           int32
user_lv_cd    int32
dtype: object

In [30]:
df_sku.dtypes

sku_id                int32
a1                    int32
a2                    int32
a3                    int32
cate                  int32
brand                 int32
comment_num           int32
has_bad_comment       int32
bad_comment_rate    float32
dtype: object

In [31]:
df_comment.head()

,dt,sku_id,comment_num,has_bad_comment,bad_comment_rate
0,2016-02-01,1000,3,1,0.0417
1,2016-02-01,10000,2,0,0.0000
2,2016-02-01,100011,4,1,0.0376
3,2016-02-01,100018,3,0,0.0000
4,2016-02-01,100020,3,0,0.0000
